In [1]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [2]:
#Initialize browser
def init_browser():
# @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
# Scrape NASA Mars News
def scrape_NASA():
    browser=init_browser()
    
    #visit the NASA Mars News Site](https://mars.nasa.gov/news/
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)
    
    #scrape page into soup
    html = browser.html
    soup=BeautifulSoup(html,"html.parser")
    
    #collect the latest News Title and Paragraph Text
    latest_news = soup.find("div", class_="list_text")
    #print(latest_news)
    #news title
    news_title=latest_news.find("div",class_="content_title").text
    #print(news_title)
    #paragraph text
    news_p=latest_news.find("div",class_="article_teaser_body").text
    
    #store in dictionary
    latest_news={
        'title':news_title,
        'paragraph':news_p
    }
    
    return latest_news

In [4]:
#Scrape JPL Mars Space Images
def scrape_JPL():
    browser = init_browser()
    
    #Visit the url for JPL Featured Space Image 
    #(https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars)
    url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)
    
    #scrape page into soup
    html=browser.html
    soup=BeautifulSoup(html,"html.parser")
    
    #find and collect the featured image
    featured_image = soup.find("section",class_="centered_text clearfix main_feature primary_media_feature single")\
                     .find("article").find("a")['data-fancybox-href']
    
    featured_image_url = 'https://www.jpl.nasa.gov'+ featured_image
    
    return featured_image_url

In [5]:
#scrape Mars Weather
def scrape_mars_weather():
    browser = init_browser()
    #Visit the Mars Weather twitter account [here]
    #(https://twitter.com/marswxreport?lang=en) 
    url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url)
    
    #scrape page into soup
    html=browser.html
    soup=BeautifulSoup(html,"html.parser")
    
    #scrape the latest Mars weather tweet from the page
    mars_weather=soup.find("div",class_="ProfileTimeline")\
          .find("div",class_="content").find("p").text
    
    return mars_weather
        

In [20]:
#scrape Mars Facts
def scrape_mars_facts():
    browser=init_browser()
    
    #Visit the Mars Facts webpage 
    #(http://space-facts.com/mars/) 
    url = 'http://space-facts.com/mars/'
    browser.visit(url)
    
    #scrape page into soup
    html=browser.html
    #soup=BeautifulSoup(html,"html.parser")
    
    #scrape the table containing facts about the planet including Diameter, Mass, etc.
    # use pandas to read the table
    table=pd.read_html(html,skiprows=1)
    #print(table)
    df =pd.DataFrame(table[0])
    df.columns=['Description','value']
    
    df=df.set_index('Description')
    print(df)
    #Use Pandas to convert the data to a HTML table string
    mars_facts=df.to_html()
    
    return mars_facts

In [21]:
mars_facts=scrape_mars_facts()

                                              value
Description                                        
Polar Diameter:                            6,752 km
Mass:                 6.42 x 10^23 kg (10.7% Earth)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.52 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                  -153 to 20 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers


In [9]:
mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>NaN</th>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>NaN</th>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>NaN</th>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>NaN</th>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>NaN</th>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>NaN</th>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>NaN</th>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>NaN</th>\n      <td>NaN</td>\n    </tr>\n  </tbody>\n</table>'

In [7]:
#scrape the images from Mars Hemispheres
def scrape_mars_hemispheres():
    browser=init_browser()
    
    #Visit the USGS Astrogeology site
    #(https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) 
    url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    
    #scrape page into soup
    html=browser.html
    soup=BeautifulSoup(html,"html.parser")
    
    #find all links to hemisphere image
    all_items=soup.find("div",class_="collapsible results").find_all("div",class_="item")
    
    #initializer hemisphere_image_urls
    hemisphere_image_urls=[]
    
    for item in all_items:

        title=item.find("div",class_="description").find("a").text    
        link='https://astrogeology.usgs.gov' + item.find("a")['href']
        #visit the link and collect in soup
        browser.visit(link)
        soup2 = BeautifulSoup(browser.html,"html.parser")
        img_url=soup2.find("img",class_='wide-image')['src']
        
        dict_item={'title':title,
                  'img_url':'https://astrogeology.usgs.gov'+ img_url}
        
        #add dict to the list
        hemisphere_image_urls.append(dict_item)
    
    return hemisphere_image_urls

In [43]:
NASA_data = scrape_NASA()
JPL_data=scrape_JPL()
mars_weather=scrape_mars_weather()
mars_facts=scrape_mars_facts()
mars_hemispheres=scrape_mars_hemispheres()

In [44]:
NASA_data

{'title': 'Scientists to Debate Landing Site for Next Mars Rover',
 'paragraph': "Hundreds of scientists and Mars-exploration enthusiasts gather this week to present, discuss and deliberate the future landing site for NASA's next Red Planet. "}

In [45]:
JPL_data

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18614_ip.jpg'

In [46]:
mars_weather

'And finally - a stack of all the images - you can see so much detail here https://dougellison.smugmug.com/Machines/Space-X-SAOCOM-1A/i-zbJzq8L\xa0…pic.twitter.com/9nLwAH3oZQ'

In [47]:
mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n

In [48]:
mars_hemispheres

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]